<a href="https://colab.research.google.com/github/otumina-collab/POO-Actividad_N-_6/blob/main/Actividad_N%C2%B06_8_4_POO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Actividad N°6 - 8.4. POO
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import json
import os

#Definición de Clases Empleado y ListaEmpleados
# Estas clases son necesarias para el funcionamiento del sistema de nómina interactivo.
class Empleado:
    def __init__(self, nombre, apellido, cargo, genero, salario_dia, dias_trabajados, otros_ingresos, pagos_salud, aporte_pensiones):
        self.nombre = nombre
        self.apellido = apellido
        self.cargo = cargo
        self.genero = genero
        self.salario_dia = salario_dia
        self.dias_trabajados = dias_trabajados
        self.otros_ingresos = otros_ingresos
        self.pagos_salud = pagos_salud
        self.aporte_pensiones = aporte_pensiones

    def calcular_salario_bruto(self):
        return (self.salario_dia * self.dias_trabajados) + self.otros_ingresos

    def calcular_deducciones(self):
        return self.pagos_salud + self.aporte_pensiones

    def calcular_salario_neto(self):
        return self.calcular_salario_bruto() - self.calcular_deducciones()

    def to_dict(self):
        return {
            'nombre': self.nombre,
            'apellido': self.apellido,
            'cargo': self.cargo,
            'genero': self.genero,
            'salario_dia': self.salario_dia,
            'dias_trabajados': self.dias_trabajados,
            'otros_ingresos': self.otros_ingresos,
            'pagos_salud': self.pagos_salud,
            'aporte_pensiones': self.aporte_pensiones
        }

    @staticmethod
    def from_dict(data):
        return Empleado(
            data['nombre'], data['apellido'], data['cargo'], data['genero'],
            data['salario_dia'], data['dias_trabajados'], data['otros_ingresos'],
            data['pagos_salud'], data['aporte_pensiones']
        )

    def __str__(self):
        return (f"  Nombre Completo: {self.nombre} {self.apellido}\n"
                f"  Cargo: {self.cargo}\n"
                f"  Género: {self.genero}\n"
                f"  Salario por Día: {self.salario_dia:,.2f}\n"
                f"  Días Trabajados: {self.dias_trabajados}\n"
                f"  Otros Ingresos: {self.otros_ingresos:,.2f}\n"
                f"  Pagos Salud: {self.pagos_salud:,.2f}\n"
                f"  Aporte Pensiones: {self.aporte_pensiones:,.2f}")

class ListaEmpleados:
    def __init__(self):
        self.empleados = []
        self.file_name = "empleados.json"

    def agregar_empleado(self, empleado):
        self.empleados.append(empleado)
        self.guardar_empleados_json()

    def obtener_resumen_nomina(self):
        if not self.empleados:
            return "No hay empleados registrados.\n"

        resumen = "========== Resumen General de Nómina ==========\n"
        total_salario_neto_general = 0
        for i, empleado in enumerate(self.empleados):
            resumen += f" Detalles del Empleado {i+1}"
            resumen += str(empleado) + "\n"

            resumen += "--------------------------------------\n"
            total_salario_neto_general += empleado.calcular_salario_neto()
        resumen += f"\n### Total Salario Neto Pagado en General: {total_salario_neto_general:,.2f} ###\n"
        resumen += "==================================================\n"
        return resumen

    def clear_all_employees(self):
        self.empleados = []
        self.guardar_empleados_json()

    def eliminar_empleado_por_nombre_completo(self, full_name):
        empleados_a_mantener = []
        eliminado = False
        for emp in self.empleados:
            if f"{emp.nombre} {emp.apellido}" == full_name and not eliminado:
                eliminado = True
            else:
                empleados_a_mantener.append(emp)
        if eliminado:
            self.empleados = empleados_a_mantener
            self.guardar_empleados_json()
            return True
        return False

    def guardar_empleados_json(self):
        try:
            with open(self.file_name, 'w', encoding='utf-8') as f:
                json.dump([emp.to_dict() for emp in self.empleados], f, indent=4, ensure_ascii=False)

        except Exception as e:
            print(f"Error al guardar empleados en JSON: {e}")

    def cargar_empleados_json(self):
        self.empleados = []
        if os.path.exists(self.file_name):
            try:
                with open(self.file_name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    for emp_data in data:
                        self.empleados.append(Empleado.from_dict(emp_data))
                print(f"Empleados cargados exitosamente desde '{self.file_name}'.")
            except json.JSONDecodeError:
                print(f"El archivo '{self.file_name}' no contiene JSON válido. Creando uno nuevo.")
                self.guardar_empleados_json()
            except Exception as e:
                print(f"Error al cargar empleados desde JSON: {e}")
        else:
            print(f"No se encontró el archivo '{self.file_name}'. Iniciando con lista de empleados vacía.")

mi_nomina_interactiva = ListaEmpleados()
mi_nomina_interactiva.cargar_empleados_json()
nombre_input = widgets.Text(description='Nombres:')
apellido_input = widgets.Text(description='Apellidos:')
cargo_input = widgets.Dropdown(
    options=[
        'Rector(a)',
        'Vicerrector(a)',
        'Decano(a)',
        'Coordinador(a) de programa',
        'Profesor(a)',
        'Investigador(a)',
        'Auxiliar de docencia',
        'Monitor académico',
        'Coordinador de laboratorio',
        'Bibliotecario(a)',
        'Secretaria(o)',
        'Asistente administrativo',
        'Gestor de admisiones y registro',
        'Contador público',
        'Analista financiero',
        'Psicólogo',
        'Trabajador(a) social',
        'Personal de aseo y mantenimiento',
        'Jardinero',
        'Personal de vigilancia',
        'Conductor',
        'Técnico de infraestructura',
        'Comunicador y diseñador',
        'Fotógrafo',
        'Encargado de mercadeo y relaciones públicas'
    ],
    description='Cargo:',
    disabled=False
)
genero_input = widgets.RadioButtons(
    options=['Masculino', 'Femenino'],
    description='Género:',
    disabled=False
)
salario_dia_input = widgets.FloatText(description='Salario/Día:')
dias_trabajados_input = widgets.IntText(description='Días Trabajados:', style={'description_width': 'initial'})
otros_ingresos_input = widgets.FloatText(description='Otros Ingresos:', style={'description_width': 'initial'})
pagos_salud_input = widgets.FloatText(description='Pagos Salud:', style={'description_width': 'initial'})
aporte_pensiones_input = widgets.FloatText(description='Aporte Pensiones:', style={'description_width': 'initial'})

# Agrupar los widgets de entrada en un VBox
input_widgets_box = widgets.VBox([
    nombre_input,
    apellido_input,
    cargo_input,
    genero_input,
    salario_dia_input,
    dias_trabajados_input,
    otros_ingresos_input,
    pagos_salud_input,
    aporte_pensiones_input
])

#Botones de Acción Internos (para secciones)
add_employee_submit_button = widgets.Button(description='Añadir Empleado', button_style='warning')
save_payroll_file_button = widgets.Button(description='Guardar Archivo de Nómina', button_style='primary')
clear_fields_button = widgets.Button(description='Limpiar Campos', button_style='warning')
calcular_nomina_button = widgets.Button(description='Calcular Nómina', button_style='warning')
clear_payroll_button = widgets.Button(description='Eliminar Toda la Nómina', button_style='danger')

#New Widgets for Individual Employee Deletion
employee_to_delete_dropdown = widgets.Dropdown(
    options=[],
    description='Seleccionar Empleado:',
    disabled=False
)
delete_selected_employee_button = widgets.Button(description='Eliminar Empleado Seleccionado', button_style='danger')
delete_single_employee_output_area = widgets.Output()


#Áreas de Salida específicas para cada sección y general
general_output_area = widgets.Output()

view_payroll_output_area = widgets.Output()

#Definición de Secciones de Contenido
# 1. Sección "Agregar Empleado"
add_employee_section = widgets.VBox([
    input_widgets_box,
    widgets.HBox([add_employee_submit_button, clear_fields_button, calcular_nomina_button]),
    general_output_area
], layout={'border': '1px solid lightgray', 'padding': '10px', 'display': 'flex'})

# 2. Sección "Ver Nómina"
view_payroll_section = widgets.VBox([
    widgets.VBox([
        view_payroll_output_area
    ], layout={'border': '1px solid #cccccc', 'padding': '5px', 'margin_bottom': '5px'}),
    clear_payroll_button
], layout={'border': '1px solid lightgray', 'padding': '5px', 'display': 'none'})

# 3. Sección "Eliminar Empleado Individual"
delete_individual_employee_section = widgets.VBox([
    employee_to_delete_dropdown,
    delete_selected_employee_button,
    delete_single_employee_output_area
], layout={'border': '1px solid lightgray', 'padding': '10px', 'display': 'none'})

# 4. Sección "Guardar Archivo"
save_file_section = widgets.VBox([
    save_payroll_file_button,
    general_output_area # Reusing general output for save operations
], layout={'border': '1px lightgray', 'padding': '10px', 'display': 'none'})

#Helper function to update employee dropdowns
def update_employee_selection_dropdown():
    employee_names = [f"{emp.nombre} {emp.apellido}" for emp in mi_nomina_interactiva.empleados]
    employee_to_delete_dropdown.options = employee_names

def clear_input_fields():
    nombre_input.value = ''
    apellido_input.value = ''
    cargo_input.value = cargo_input.options[0] if cargo_input.options else ''
    genero_input.value = 'Masculino'
    salario_dia_input.value = 0.0
    dias_trabajados_input.value = 0
    otros_ingresos_input.value = 0.0
    pagos_salud_input.value = 0.0
    aporte_pensiones_input.value = 0.0

#Funciones de Manejo de Eventos (Handlers)
def add_employee_handler(b):
    with general_output_area:
        clear_output(wait=True)
        try:
            nombre = nombre_input.value
            apellido = apellido_input.value
            cargo = cargo_input.value
            genero = genero_input.value
            salario_dia = salario_dia_input.value
            dias_trabajados = dias_trabajados_input.value
            otros_ingresos = otros_ingresos_input.value
            pagos_salud = pagos_salud_input.value
            aporte_pensiones = aporte_pensiones_input.value

            if not nombre:
                print("Error: El campo Nombres del empleado no puede estar vacío.")
                return
            if not apellido:
                print("Error: El campo Apellidos del empleado no puede estar vacío.")
                return
            if not genero:
                print("Error: El campo Género del empleado no puede estar vacío.")
                return
            if not all(isinstance(x, (int, float)) for x in [salario_dia, dias_trabajados, otros_ingresos, pagos_salud, aporte_pensiones]):
                print("Error: Todos los campos numéricos deben ser válidos.")
                return

            nuevo_empleado = Empleado(
                nombre, apellido, cargo, genero, salario_dia, dias_trabajados, otros_ingresos,
                pagos_salud, aporte_pensiones
            )
            mi_nomina_interactiva.agregar_empleado(nuevo_empleado)
            print(f"Empleado '{nuevo_empleado.nombre} {nuevo_empleado.apellido}' añadido con éxito.")

            # Update payroll summary and employee selection dropdowns
            with view_payroll_output_area:
                clear_output(wait=True)
                print(mi_nomina_interactiva.obtener_resumen_nomina())
            update_employee_selection_dropdown()

            clear_input_fields()

        except Exception as e:
            print(f"Error al añadir empleado: {e}")

def save_payroll_handler(b):
    with general_output_area:
        clear_output(wait=True)
        output_file_name = "resumen_nomina.txt"
        json_file_name = mi_nomina_interactiva.file_name

        if not mi_nomina_interactiva.empleados:
            print("No hay datos de nómina para guardar.")
            return

        try:
            # Guardar el resumen de nómina en un archivo de texto (comportamiento original)
            summary_content = mi_nomina_interactiva.obtener_resumen_nomina()
            with open(output_file_name, "w", encoding='utf-8') as f:
                f.write(summary_content)
            print(f"Resumen de nómina guardado exitosamente en '{output_file_name}'.")

            # La lista de empleados se guarda automáticamente en JSON al añadir/eliminar.
            mi_nomina_interactiva.guardar_empleados_json()
            print(f"Datos de empleados guardados automáticamente en '{json_file_name}' para persistencia.")
            print("Puedes descargar los archivos haciendo clic en el icono de 'Archivos' (carpeta) en el panel izquierdo de Colab.")
        except Exception as e:
            print(f"Error al guardar el archivo de nómina: {e}")

def clear_fields_handler(b):
    with general_output_area:
        clear_output(wait=True)
        clear_input_fields()
        print("Campos de entrada limpiados.")

def calcular_nomina_handler(b):
    with general_output_area:
        clear_output(wait=True)
        print("Calculando nómina...")
    show_section('view_payroll')

def clear_payroll_handler(b):
    with general_output_area:
        clear_output(wait=True)
        if mi_nomina_interactiva.empleados:
            mi_nomina_interactiva.clear_all_employees()
            print("¡Nómina eliminada con éxito! Todos los empleados han sido borrados.")
            print(f"El archivo '{mi_nomina_interactiva.file_name}' ha sido actualizado para reflejar la nómina vacía.")
        else:
            print("No hay empleados para eliminar en la nómina.")
    with view_payroll_output_area:
        clear_output(wait=True)
        print(mi_nomina_interactiva.obtener_resumen_nomina())
    update_employee_selection_dropdown()
    clear_input_fields()

def delete_selected_employee_handler(b):
    with delete_single_employee_output_area:
        clear_output(wait=True)
        selected_name = employee_to_delete_dropdown.value
        if not selected_name:
            print("Error: No hay ningún empleado seleccionado para eliminar.")
            return

        confirm_delete = mi_nomina_interactiva.eliminar_empleado_por_nombre_completo(selected_name)
        if confirm_delete:
            print(f"Empleado '{selected_name}' eliminado con éxito.")
            print(f"Los datos de empleados han sido actualizados en '{mi_nomina_interactiva.file_name}'.")
        else:
            print(f"Error: No se encontró al empleado '{selected_name}' para eliminar.")

        # Update all relevant displays
        update_employee_selection_dropdown()
        with view_payroll_output_area:
            clear_output(wait=True)
            print(mi_nomina_interactiva.obtener_resumen_nomina())

# Attach handlers to internal action buttons
add_employee_submit_button.on_click(add_employee_handler)
save_payroll_file_button.on_click(save_payroll_handler)
clear_fields_button.on_click(clear_fields_handler)
calcular_nomina_button.on_click(calcular_nomina_handler)
clear_payroll_button.on_click(clear_payroll_handler)
delete_selected_employee_button.on_click(delete_selected_employee_handler)

# --- Botones de Navegación Principales ---
main_add_button = widgets.Button(description='Agregar Empleado', button_style='success')
main_view_button = widgets.Button(description='Ver Nómina', button_style='success')
main_delete_single_button = widgets.Button(description='Eliminar Empleado', button_style='success')
main_save_button = widgets.Button(description='Guardar Archivo', button_style='success')

# Create the Menu HTML widget with button-like dimensions
menu_box_widget = widgets.HTML(
    value="<h2 style='text-align: center; padding: 5px; background-color: #28a745; color: white;'>Menu</h2>",
    layout=widgets.Layout(
        border='1px solid #28a745', # Changed border color to green, matching button style
        width='180px', # Approximate button width
        margin='0px 5px 0px 0px' # Margin to separate from next button
    )
)

# Agrupar botones de navegación principales, incluyendo el nuevo widget de menú
main_buttons_box = widgets.HBox([
    menu_box_widget,
    main_add_button,
    main_view_button,
    main_delete_single_button,
    main_save_button
])

# --- Función para mostrar/ocultar secciones ---
def show_section(section_name):
    # Hide all sections
    add_employee_section.layout.display = 'none'
    view_payroll_section.layout.display = 'none'
    delete_individual_employee_section.layout.display = 'none'
    save_file_section.layout.display = 'none'

    # Clear general output area when switching sections
    with general_output_area:
        clear_output(wait=True)
    with delete_single_employee_output_area:
        clear_output(wait=True)

    # Show the selected section
    if section_name == 'add_employee':
        add_employee_section.layout.display = 'flex'
    elif section_name == 'view_payroll':
        view_payroll_section.layout.display = 'flex'
        with view_payroll_output_area:
            clear_output(wait=True)
            print(mi_nomina_interactiva.obtener_resumen_nomina())
    elif section_name == 'delete_single_employee':
        delete_individual_employee_section.layout.display = 'flex'
        update_employee_selection_dropdown()
        if not mi_nomina_interactiva.empleados:
            with delete_single_employee_output_area:
                print("No hay empleados para eliminar individualmente.")
    elif section_name == 'save_file':
        save_file_section.layout.display = 'flex'

#Attach Handlers to Main Navigation Buttons
main_add_button.on_click(lambda b: show_section('add_employee'))
main_view_button.on_click(lambda b: show_section('view_payroll'))
main_delete_single_button.on_click(lambda b: show_section('delete_single_employee'))
main_save_button.on_click(lambda b: show_section('save_file'))

#Display the Main Interface
display(HTML("<h2 style='text-align: center; border: 2px solid white; padding: 10px; margin-bottom: 10px;'>Nomina</h2>"))

main_interface_box = widgets.VBox([
    main_buttons_box,
    widgets.VBox([
        add_employee_section,
        view_payroll_section,
        delete_individual_employee_section,
        save_file_section
    ])
], layout={'border': '3px solid white', 'padding': '10px'})
display(main_interface_box)

# Initially show the 'Add Employee' section
show_section('add_employee')

update_employee_selection_dropdown()
with view_payroll_output_area:
    clear_output(wait=True)
    print(mi_nomina_interactiva.obtener_resumen_nomina())

No se encontró el archivo 'empleados.json'. Iniciando con lista de empleados vacía.
